<a href="https://colab.research.google.com/github/iamakashkumar09/Solar_Power_Analysis_ML/blob/main/Recurrent_Neural_Network_(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [94]:
data = pd.read_csv('powerPlantDataBerkeley.csv')

In [95]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(data.mean(), inplace=True)

In [96]:
# identifying features and target
features = [
    "Day of Year","Year","Month","Day","First Hour of Period","Is Daylight","Distance to Solar Noon", "Average Temperature (Day)","Average Wind Direction (Day)", "Average Wind Speed (Day)",
    "Sky Cover", "Visibility", "Relative Humidity", "Average Wind Speed (Period)",
    "Average Barometric Pressure (Period)"
]
target = "Power Generated"

X = data[features].values
y = data[target].values

In [97]:
# normalization
scaler_X = MinMaxScaler()
X_new = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_new = scaler_y.fit_transform(y.reshape(-1,1))

In [98]:
# implimenting inputs of RNN in the form (input,forget,output)
X_new = X_new.reshape(X_new.shape[0], 1, X_new.shape[1])

In [99]:
X_train,X_test,y_train,y_test = train_test_split(X_new,y_new,test_size=0.2,random_state=42)

In [100]:
model = Sequential([
    LSTM(50,return_sequences=True,input_shape=(1, len(features))),
    Dropout(0.2),
    LSTM(50,return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1,activation='relu')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [101]:
# compile the model
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [102]:
# training
history = model.fit(X_train,y_train,epochs=30,batch_size=16,validation_data=(X_test,y_test))

Epoch 1/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0690 - val_loss: 0.0179
Epoch 2/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0148 - val_loss: 0.0176
Epoch 3/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0119 - val_loss: 0.0135
Epoch 4/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0123 - val_loss: 0.0143
Epoch 5/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0125 - val_loss: 0.0128
Epoch 6/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0130 - val_loss: 0.0130
Epoch 7/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0123 - val_loss: 0.0134
Epoch 8/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0135 - val_loss: 0.0143
Epoch 9/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0120 - val_loss: 0.0126
Epoch 10/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0108 - val_loss: 0.0180
Epoch 11/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0122 - val_loss: 0.0129
Epoch 12/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

In [103]:
print("Training loss : ",history.history['loss'][-1])

Training loss :  0.008878560736775398


In [104]:
# Evaluating
test_loss = model.evaluate(X_test, y_test)
print("test loss : ",test_loss)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0113 
test loss :  0.011539102531969547


In [105]:
y_pred = model.predict(X_test)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [106]:
y_pred_new = scaler_y.inverse_transform(y_pred)
y_test_new = scaler_y.inverse_transform(y_test)

print("predictions : ",y_pred_new[:10])

predictions :  [[21719.004 ]
 [ 3994.9092]
 [    0.    ]
 [ 2902.2437]
 [ 2526.2102]
 [25825.219 ]
 [    0.    ]
 [    0.    ]
 [    0.    ]
 [    0.    ]]
